In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model,Sequential
from keras.layers import *
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.math import l2_normalize
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from gensim.models import KeyedVectors
from keras import layers


In [ ]:
!wget -P download -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
word2vec = KeyedVectors.load_word2vec_format('download/GoogleNews-vectors-negative300.bin.gz', binary=True)

--2020-12-02 07:09:03--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.128.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.128.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘download/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  46.1MB/s    in 36s     

2020-12-02 07:09:39 (43.6 MB/s) - ‘download/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
(xtrain,ytrain),(xtest,ytest)=imdb.load_data(num_words=5000) 

17465344/17464789 [==============================] - 1s 0us/step


In [ ]:
maxlen=500
vocab_size=5000
emb_dimension=300
xtrain=pad_sequences(xtrain,maxlen=maxlen,padding='post')
xtest=pad_sequences(xtest,maxlen=maxlen,padding='post')

In [ ]:
word_idx=imdb.get_word_index() #getting vocab from imdb data

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
embedding_matrix = np.zeros((vocab_size,emb_dimension))
for word, i in word_idx.items():
    if word in word2vec.vocab and i<vocab_size:
        embedding_matrix[i] = word2vec.word_vec(word)

In [ ]:
# channel 1
inputs1 = Input(shape=(maxlen,),)
embedding1=Embedding(vocab_size,emb_dimension,weights=[embedding_matrix])(inputs1)
conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)

# channel 2
conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding1)
drop2 = Dropout(0.5)(conv2)
pool2 = MaxPooling1D(pool_size=2)(drop2)
flat2 = Flatten()(pool2)

# channel 3
conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding1)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)

# merge
merged = concatenate([flat1, flat2, flat3])
# interpretation
dense1 = Dense(10, activation='relu')(merged)
outputs = Dense(1, activation='sigmoid')(dense1)
model = Model(inputs=inputs1, outputs=outputs)
# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize
print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 500, 300)     1500000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 497, 32)      38432       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 495, 32)      57632       embedding[0][0]                  
_______________________________________________________________________________________

In [ ]:
plot_model(model, show_shapes=True, to_file='multichannel.png')

In [ ]:
model.fit(xtrain, ytrain, batch_size=128, epochs=15, validation_data=(xtest, ytest))

Epoch 1/15
196/196 [==============================] - 574s 3s/step - loss: 0.5594 - accuracy: 0.6729 - val_loss: 0.3671 - val_accuracy: 0.8422
Epoch 2/15
196/196 [==============================] - 573s 3s/step - loss: 0.2623 - accuracy: 0.8949 - val_loss: 0.2954 - val_accuracy: 0.8781
Epoch 3/15
196/196 [==============================] - 574s 3s/step - loss: 0.1828 - accuracy: 0.9299 - val_loss: 0.2875 - val_accuracy: 0.8786
Epoch 4/15
196/196 [==============================] - 577s 3s/step - loss: 0.1202 - accuracy: 0.9578 - val_loss: 0.3645 - val_accuracy: 0.8606
Epoch 5/15
196/196 [==============================] - 577s 3s/step - loss: 0.0665 - accuracy: 0.9788 - val_loss: 0.3493 - val_accuracy: 0.8692
Epoch 6/15
196/196 [==============================] - 583s 3s/step - loss: 0.0380 - accuracy: 0.9889 - val_loss: 0.4213 - val_accuracy: 0.8680
Epoch 7/15
196/196 [==============================] - 580s 3s/step - loss: 0.0265 - accuracy: 0.9924 - val_loss: 0.4365 - val_accuracy: 0.8679